In [1]:
# Installation de GraphFrames et configuration Spark
!apt-get update
!apt-get install -y openjdk-11-jdk-headless




Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

In [3]:
!pip install pyspark==3.2.1
!pip install graphframes

Nousallons utiliser un Dataset composé de deux fichiers :

  Lepremier fichier "station_data.csv"représente des stations.
 
  Le deuxième fichier "trip_data.csv" représente des voyages (trips) à travers des 
vélos.

 1) Créer unDataFrame pourchaque fichier.

In [4]:
from pyspark.sql import SparkSession

# Création de la session Spark
spark = SparkSession.builder \
    .appName("GraphFrames TP") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark

In [5]:
from google.colab import files

print("Téléchargez le fichier station_data.csv")
uploaded_stations = files.upload()

print("Téléchargez le fichier trip_data.csv")
uploaded_trips = files.upload()

# Affichage des fichiers uploadés
print("Fichiers uploadés:")
for filename in uploaded_stations.keys():
    print(f"- {filename}")
for filename in uploaded_trips.keys():
    print(f"- {filename}")

Téléchargez le fichier station_data.csv


Saving station_data.csv to station_data.csv
Téléchargez le fichier trip_data.csv


Saving trip_data.csv to trip_data.csv
Fichiers uploadés:
- station_data.csv
- trip_data.csv


In [6]:
# Chargement des DataFrames
stations_df = spark.read.csv("station_data.csv", header=True, inferSchema=True)
trips_df = spark.read.csv("trip_data.csv", header=True, inferSchema=True)

# Vérification
print("Schéma des stations:")
stations_df.printSchema()
print("Aperçu des stations:")
stations_df.show(5)

print("\nSchéma des voyages:")
trips_df.printSchema()
print("Aperçu des voyages:")
trips_df.show(5)

Schéma des stations:
root
 |-- station_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dockcount: integer (nullable = true)
 |-- landmark: string (nullable = true)
 |-- installation: string (nullable = true)

Aperçu des stations:
+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                name|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|     

In [7]:
from graphframes import GraphFrame

# 2) Renommer la colonne name en id pour les sommets
vertices_df = stations_df.withColumnRenamed("name", "id")

# 3) Renommer les colonnes pour les arcs
edges_df = trips_df.withColumnRenamed("Start Station", "src") \
                   .withColumnRenamed("End Station", "dst")

print("Sommets préparés:")
vertices_df.show(5)
print("\nArcs préparés:")
edges_df.show(5)

Sommets préparés:
+----------+--------------------+---------+-----------+---------+--------+------------+
|station_id|                  id|      lat|       long|dockcount|landmark|installation|
+----------+--------------------+---------+-----------+---------+--------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|       15|San Jose|    8/7/2013|
+----------+--------------------+---------+-----------+---------+--------+------------+
only showing top 5 rows


Arcs préparés:
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+------

In [8]:
# 4) Création du GraphFrame
graph = GraphFrame(vertices_df, edges_df)

print("=== GRAPHE CRÉÉ ===")
print(f"Nombre de sommets: {graph.vertices.count()}")
print(f"Nombre d'arcs: {graph.edges.count()}")

=== GRAPHE CRÉÉ ===
Nombre de sommets: 70
Nombre d'arcs: 99


In [9]:
# 5) Nombre de voyages entre chaque source et destination
voyages_par_trajet = graph.edges.groupBy("src", "dst") \
    .count() \
    .orderBy("count", ascending=False)

print("Voyages par trajet (triés par nombre décroissant):")
voyages_par_trajet.show(10)

Voyages par trajet (triés par nombre décroissant):
+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|San Francisco Cal...|    3|
|San Francisco Cal...|  Powell Street BART|    2|
|     2nd at Townsend|   Market at Sansome|    2|
|   Market at Sansome|Broadway St at Ba...|    2|
|San Francisco Cal...|   2nd at South Park|    2|
|     Spear at Folsom|     2nd at Townsend|    2|
|    Davis at Jackson|Embarcadero at Sa...|    2|
|   Steuart at Market|San Francisco Cal...|    2|
|       Market at 4th|San Francisco Cal...|    2|
+--------------------+--------------------+-----+
only showing top 10 rows



In [10]:
# 6) Voyages commençant ou se terminant à 'Townsend at 7th'
voyages_townsend = graph.edges.filter( # edges => dataframe des arcs
    (graph.edges.src == "Townsend at 7th") |
    (graph.edges.dst == "Townsend at 7th")
).groupBy("src", "dst") \
 .count() \
 .orderBy("count", ascending=False)

print("Voyages commençant ou se terminant à 'Townsend at 7th':")
voyages_townsend.show()

Voyages commençant ou se terminant à 'Townsend at 7th':
+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|     Townsend at 7th|    1|
|     Townsend at 7th|Harry Bridges Pla...|    1|
|     Townsend at 7th|     Spear at Folsom|    1|
|     Spear at Folsom|     Townsend at 7th|    1|
|       Howard at 2nd|     Townsend at 7th|    1|
+--------------------+--------------------+-----+



In [ ]:
# 7) Stations jamais destination depuis 'Spear at Folsom'
stations_depart_spear = graph.edges.filter(
    graph.edges.src == "Spear at Folsom"
).select("dst").distinct()

toutes_stations = graph.vertices.select("id")

stations_jamais_destination = toutes_stations.join(
    stations_depart_spear,
    toutes_stations.id == stations_depart_spear.dst,
    "left_anti"  # garder juste le coté gauche qui n'a pas de correspondance avec le coté droit
)

print("Stations jamais destination depuis 'Spear at Folsom':")
stations_jamais_destination.show()

Stations jamais destination depuis 'Spear at Folsom':
+--------------------+
|                  id|
+--------------------+
|San Jose Diridon ...|
|San Jose Civic Ce...|
|Santa Clara at Al...|
|    Adobe on Almaden|
|    San Pedro Square|
|Paseo de San Antonio|
| San Salvador at 1st|
|           Japantown|
|  San Jose City Hall|
|         MLK Library|
|SJSU 4th at San C...|
|       St James Park|
|Arena Green / SAP...|
|SJSU - San Salvad...|
|   Franklin at Maple|
|Redwood City Calt...|
|San Mateo County ...|
|Redwood City Publ...|
|Stanford in Redwo...|
|Redwood City Medi...|
+--------------------+
only showing top 20 rows



In [11]:
# 8) Station avec le nombre maximum de voyages entrants
station_max_entrants = graph.inDegrees.orderBy("inDegree", ascending=False)

print("Stations par nombre de voyages entrants:")
station_max_entrants.show()

station_max = station_max_entrants.first()
print(f"\nStation avec le plus de voyages entrants: {station_max['id']}")
print(f"Nombre de voyages entrants: {station_max['inDegree']}")

Stations par nombre de voyages entrants:
+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|San Francisco Cal...|       9|
|     2nd at Townsend|       8|
|     Townsend at 7th|       7|
|Embarcadero at Sa...|       7|
|San Francisco Cal...|       7|
|   Market at Sansome|       5|
|     Spear at Folsom|       5|
|   Steuart at Market|       5|
|   2nd at South Park|       4|
|  Powell Street BART|       3|
|Broadway St at Ba...|       3|
|Grant Avenue at C...|       3|
|Embarcadero at Br...|       3|
|Harry Bridges Pla...|       3|
|       Market at 4th|       3|
|Powell at Post (U...|       2|
|Temporary Transba...|       2|
|San Francisco Cit...|       2|
|       5th at Howard|       2|
|South Van Ness at...|       2|
+--------------------+--------+
only showing top 20 rows


Station avec le plus de voyages entrants: San Francisco Caltrain 2 (330 Townsend)
Nombre de voyages entrants: 9


In [12]:
# 9) Voyage avec la plus grande durée
voyage_plus_long = trips_df.orderBy("Duration", ascending=False).first()

print("=== VOYAGE AVEC LA PLUS GRANDE DURÉE ===")
print(f"Trip ID: {voyage_plus_long['Trip ID']}")
print(f"Station de départ: {voyage_plus_long['Start Station']}")
print(f"Station d'arrivée: {voyage_plus_long['End Station']}")
print(f"Durée: {voyage_plus_long['Duration']} secondes")

=== VOYAGE AVEC LA PLUS GRANDE DURÉE ===
Trip ID: 913386
Station de départ: Embarcadero at Bryant
Station d'arrivée: Harry Bridges Plaza (Ferry Building)
Durée: 1808 secondes


In [ ]:
# 10) Créer un sous-graphe qui ne contient que les voyages qui se commencent ou se terminent à ‘Townsend at 7th’.

# 1/on garde seulement : les voyages qui commencent à "Townsend at 7th" (src) ou les voyages qui finissent à "Townsend at 7th" (dst).
edges_townsend = graph.edges.filter(
    (graph.edges.src == "Townsend at 7th") |
    (graph.edges.dst == "Townsend at 7th")
)

#2/ on cree le dataframe des sommet
vertices_ids_townsend = edges_townsend.select("src").union(
    edges_townsend.select("dst")
).distinct().withColumnRenamed("src", "id")

# on lui donne les 2 dataframe pour construire le sous graph
subgraph_townsend = GraphFrame(vertices_ids_townsend, edges_townsend)

print("=== SOUS-GRAPHE TOWNSEND AT 7TH ===")
print(f"Nombre de sommets: {subgraph_townsend.vertices.count()}")
print(f"Nombre d'arcs: {subgraph_townsend.edges.count()}")
subgraph_townsend.edges.show()

=== SOUS-GRAPHE TOWNSEND AT 7TH ===
Nombre de sommets: 6
Nombre d'arcs: 9
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913434|     283|8/31/2015 21:19|San Francisco Cal...|            69|8/31/2015 21:24|     Townsend at 7th|          65|   521|     Subscriber|   94107|
| 913404|     273|8/31/2015 20:39|San Francisco Cal...|            69|8/31/2015 20:44|     Townsend at 7th|          65|   287|     Subscriber|   94107|
| 913382|     645|8/31/2015 20:20|     Townsend at 7th|            65|8/31/2015 20:31|Harry Bridges Pla...|          50|   270|     Subscriber|   94945|
| 913379

In [13]:
 #11) Retourner tous les chemins qui forment un motif en "triangle" entre trois stations.
print("=== RECHERCHE DE TRIANGLES ===")

triangles = graph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)")

print(f"Nombre de triangles trouvés: {triangles.count()}")
if triangles.count() > 0:
    triangles.show()
else:
    print("Aucun triangle trouvé dans le graphe.")

=== RECHERCHE DE TRIANGLES ===
Nombre de triangles trouvés: 42
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  ab|                   b|                  bc|                   c|                  ca|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|{60, Embarcadero ...|{913450, 255, 8/3...|{74, Steuart at M...|{913315, 518, 8/3...|{61, 2nd at Towns...|{913383, 1435, 8/...|
|{60, Embarcadero ...|{913443, 691, 8/3...|{77, Market at Sa...|{913403, 497, 8/3...|{61, 2nd at Towns...|{913383, 1435, 8/...|
|{69, San Francisc...|{913434, 283, 8/3...|{65, Townsend at ...|{913371, 663, 8/3...|{49, Spear at Fol...|{913432, 703, 8/3...|
|{49, Spear at Fol...|{913432, 703, 8/3...|{69, San Francisc...|{913308, 185, 8/3...|{61, 2nd at Towns...|{913297, 501, 8/3...|
|{49, Spear at Fol...|{913432, 703, 8/3..

In [15]:
#Retourner tous les chemins qui passent par trois sommets et qui commencent à partir

motif_chain = graph.find("(a)-[e1]->(b); (b)-[e2]->(c)").filter("a.id != c.id").filter("a.id = 'Townsend at 7th'")
motif_chain.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  e1|                   b|                  e2|                   c|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|{65, Townsend at ...|{913382, 645, 8/3...|{50, Harry Bridge...|{913349, 559, 8/3...|{61, 2nd at Towns...|
|{65, Townsend at ...|{913382, 645, 8/3...|{50, Harry Bridge...|{913415, 274, 8/3...|{54, Embarcadero ...|
|{65, Townsend at ...|{913382, 645, 8/3...|{50, Harry Bridge...|{913460, 765, 8/3...|{70, San Francisc...|
|{65, Townsend at ...|{913371, 663, 8/3...|{49, Spear at Fol...|{913290, 273, 8/3...|{61, 2nd at Towns...|
|{65, Townsend at ...|{913371, 663, 8/3...|{49, Spear at Fol...|{913344, 326, 8/3...|{42, Davis at Jac...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

